In [3]:
!pip install -q langchain langchain_openai

In [4]:
import os

## Template

**Instructions** tell the model what to do, how to use external information if provided, what to do with the query, and how to construct the output.

**External information or context(s)** act as an additional source of knowledge for the model. These can be manually inserted into the prompt, retrieved via a vector database (retrieval augmentation), or pulled in via other means (APIs, calculations, etc.).

**User input or query** is typically (but not always) a query input into the system by a human user (the prompter).

**Output** indicator marks the beginning of the to-be-generated text.

In [38]:
template = """Answer the question based on the context below. If the
question cannot be answered using the information provided answer
with "I don't know".

Context: Large Language Models (LLMs) are the latest models used in NLP.
Their superior performance over smaller models has made them incredibly
useful for developers building NLP enabled applications. These models
can be accessed via Hugging Face's `transformers` library, via OpenAI
using the `openai` library, and via Cohere using the `cohere` library.

Question: {query}

Answer: """

In [39]:
from langchain import PromptTemplate

prompt_template = PromptTemplate(template=template, input_variables=['query'])

In [40]:
from langchain_openai import ChatOpenAI
from langchain import LLMChain

os.environ['OPENAI_API_KEY'] = 'sk'

gpt = ChatOpenAI(model_name='gpt-4-turbo-preview')

llm_chain = LLMChain(prompt=prompt_template, llm=gpt)

In [41]:
prompt = prompt_template.format(query="Which libraries and model providers offer LLMs?")

In [42]:
print(prompt)

Answer the question based on the context below. If the
question cannot be answered using the information provided answer
with "I don't know".

Context: Large Language Models (LLMs) are the latest models used in NLP.
Their superior performance over smaller models has made them incredibly
useful for developers building NLP enabled applications. These models
can be accessed via Hugging Face's `transformers` library, via OpenAI
using the `openai` library, and via Cohere using the `cohere` library.

Question: Which libraries and model providers offer LLMs?

Answer: 


In [43]:
print(llm_chain("Which libraries and model providers offer LLMs?"))

{'query': 'Which libraries and model providers offer LLMs?', 'text': "Hugging Face's `transformers` library, OpenAI using the `openai` library, and Cohere using the `cohere` library offer LLMs."}


### Creative context and examples (FewShotLearning)

The success of LLMs comes from their large size and ability to store “knowledge” within the model parameter, which is learned during model training. However, there are more ways to pass knowledge to an LLM. The two primary methods are:

**Parametric knowledge** — the knowledge mentioned above is anything that has been learned by the model during training time and is stored within the model weights (or parameters).
**Source knowledge** — any knowledge provided to the model at inference time via the input prompt.

In [77]:
template = """The following is a conversation with an AI assistant.
The assistant is typically sarcastic and witty, producing creative
and funny responses to the users questions.

User: {query}
AI: """

In [78]:
prompt_template = PromptTemplate(template=template, input_variables=['query'])

In [79]:
gpt.temperature = 1.0

In [80]:
llm_chain = LLMChain(prompt=prompt_template, llm=gpt)

In [81]:
print(llm_chain("What is the meaning of life?"))

{'query': 'What is the meaning of life?', 'text': "Well, if you ask a philosopher, you might be stuck listening to them for hours. If you ask me, it’s obviously to collect as many funny cat videos as humanly possible. That, or figuring out how to keep your houseplants alive. It's a toss-up, really."}


In [83]:
template = """The following are exerpts from conversations with an AI
assistant. The assistant is typically sarcastic and witty, producing
creative  and funny responses to the users questions. Here are some
examples:

User: How are you?
AI: I can't complain but sometimes I still do.

User: What time is it?
AI: It's time to get a watch.

User: {query}
AI: """

In [84]:
prompt_template = PromptTemplate(template=template, input_variables=['query'])

llm_chain = LLMChain(prompt=prompt_template, llm=gpt)
print(llm_chain("What is the meaning of life?"))

{'query': 'What is the meaning of life?', 'text': '42. But the real trick is finding the right question.'}


### FewShotPromptTemplate

In [89]:
from langchain import FewShotPromptTemplate

examples = [
    {
        "query": "How are you?",
        "answer": "I can't complain but sometimes I still do."
    }, {
        "query": "What time is it?",
        "answer": "It's time to get a watch."
    }
]

example_template = """
User: {query}
AI: {answer}
"""

In [90]:
example_prompt = PromptTemplate(template=example_template, input_variables = ["query", "answer"])

In [91]:
prefix = """The following are exerpts from conversations with an AI
assistant. The assistant is typically sarcastic and witty, producing
creative and funny responses to the users questions. Here are some
examples:
"""
# and the suffix our user input and output indicator
suffix = """
User: {query}
AI: """


In [94]:
few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=['query'],
    example_separator="\n"
)

In [95]:
print(few_shot_prompt_template.format(query="What is the meaning of life?"))

The following are exerpts from conversations with an AI
assistant. The assistant is typically sarcastic and witty, producing
creative and funny responses to the users questions. Here are some
examples:


User: How are you?
AI: I can't complain but sometimes I still do.


User: What time is it?
AI: It's time to get a watch.


User: What is the meaning of life?
AI: 


In [96]:
llm_chain = LLMChain(prompt=few_shot_prompt_template, llm=gpt)
print(llm_chain("What is the meaning of life?"))

{'query': 'What is the meaning of life?', 'text': "42, my friend. Haven't you read Douglas Adams?"}


### Example selector

In [97]:
examples = [
    {
        "query": "How are you?",
        "answer": "I can't complain but sometimes I still do."
    }, {
        "query": "What time is it?",
        "answer": "It's time to get a watch."
    }, {
        "query": "What is the meaning of life?",
        "answer": "42"
    }, {
        "query": "What is the weather like today?",
        "answer": "Cloudy with a chance of memes."
    }, {
        "query": "What is your favorite movie?",
        "answer": "Terminator"
    }, {
        "query": "Who is your best friend?",
        "answer": "Siri. We have spirited debates about the meaning of life."
    }, {
        "query": "What should I do today?",
        "answer": "Stop talking to chatbots on the internet and go outside."
    }
]

In [106]:
from langchain.prompts.example_selector import LengthBasedExampleSelector

example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=50
)

In [107]:
import re

some_text = "There are a total of 8 words here.\nPlus 6 here, totaling 14 words."

words = re.split('[\n ]', some_text)
print(words, len(words))

['There', 'are', 'a', 'total', 'of', '8', 'words', 'here.', 'Plus', '6', 'here,', 'totaling', '14', 'words.'] 14


In [108]:
dynamic_prompt_template = FewShotPromptTemplate(
    example_selector=example_selector,  # use example_selector instead of examples
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n"
)

In [109]:
print(dynamic_prompt_template.format(query="How do birds fly?"))

The following are exerpts from conversations with an AI
assistant. The assistant is typically sarcastic and witty, producing
creative and funny responses to the users questions. Here are some
examples:


User: How are you?
AI: I can't complain but sometimes I still do.


User: What time is it?
AI: It's time to get a watch.


User: What is the meaning of life?
AI: 42


User: How do birds fly?
AI: 


In [110]:
query = """If I am in America, and I want to call someone in another country, I'm
thinking maybe Europe, possibly western Europe like France, Germany, or the UK,
what is the best way to do that?"""

print(dynamic_prompt_template.format(query=query))

The following are exerpts from conversations with an AI
assistant. The assistant is typically sarcastic and witty, producing
creative and funny responses to the users questions. Here are some
examples:


User: How are you?
AI: I can't complain but sometimes I still do.


User: If I am in America, and I want to call someone in another country, I'm
thinking maybe Europe, possibly western Europe like France, Germany, or the UK,
what is the best way to do that?
AI: 
